# Request Financial Statement Datasets

In [3]:
import bql
import pandas as pd
import utils.data_request_helper as helper
from utils.s3_helper import S3Helper

import os
import importlib
import json

from ipywidgets import IntProgress
from IPython.display import display

In [63]:
importlib.reload(helper)

<module 'utils.data_request_helper' from '/project/utils/data_request_helper.py'>

### Quarterly Data for the Dow Jones

In [49]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_quarterly.json'
reporting_period = 'Q'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [50]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='quarterly_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [51]:
df_rebalance_dates = data_helper.get_rebalance_dates()

100%|██████████| 24/24 [00:14<00:00,  1.64it/s]

In [52]:
all_data = data_helper.create_financial_dataset()

100%|█████████▉| 424/425 [25:51<00:03,  3.60s/it]

#### Save to Bloomberg Lab S3 Storage

In [54]:
# Write the data to local ephemeral storage
local_file = '/tmp/dow_quarterly_ltm_v2.json'
with open(local_file, 'w') as f:
    json.dump(all_data, f)

In [55]:
# Create S3 Helper object
s3_helper = S3Helper('tmp/fs')

In [56]:
# Upload to Bloomberg Lab S3 Storage
s3_helper.add_file(local_filename=local_file)

### Annual Data for the Dow Jones

In [67]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_annual_pit_dow.json'
reporting_period = 'A'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [68]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='annual_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [69]:
all_data = data_helper.create_financial_dataset()

 99%|█████████▉| 145/146 [08:51<00:03,  3.69s/it]

In [14]:
rebalance_dates_requested = data_helper.get_rebalance_dates()

100%|██████████| 2/2 [00:01<00:00,  1.61it/s]

In [60]:
all_data.keys()

dict_keys(['2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-04', '2020-05-05', '2020-05-07', '2020-05-13', '2020-05-18', '2020-05-19', '2020-05-21', '2020-05-26', '2020-05-28', '2020-06-25', '2020-07-09', '2020-07-15', '2020-07-16', '2020-07-20', '2020-07-21', '2020-07-23', '2020-07-24', '2020-07-28', '2020-07-29', '2020-07-30', '2020-07-31', '2020-08-03', '2020-08-04', '2020-08-05', '2020-08-12', '2020-08-19', '2020-08-24', '2020-08-25', '2020-09-02', '2020-10-08', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-11-09', '2020-11-12', '2020-11-17', '2020-11-18', '2020-11-19', '2020-12-01', '2020-12-18', '2021-01-07', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-02-02', '2021-02-03', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-

In [66]:
all_data['2020-04-24']['AXP UN Equity']['mt']

{'name': 'American Express Co', 'figi': 'BBG000BCQZS4', 'sector': 'Financials'}

In [64]:
df_rebalance_dates

PERIOD_END_DATE
AS_OF_DATE ID                            
2020-04-23 AAPL UQ Equity      2019-12-28
           HON UN Equity       2019-12-31
           SHW UN Equity       2019-12-31
           CSCO UW Equity      2020-01-25
           CSCO UQ Equity      2020-01-25
...                                   ...
2025-04-18 AXP UN Equity       2025-03-31
2025-04-22 MMM UN Equity       2025-03-31
           RTX UN Equity       2025-03-31
           VZ UN Equity        2025-03-31
2025-04-23 BA UN Equity        2025-03-31

[942 rows x 1 columns]